# The Task
### Determine the EC50 and hill coefficient for an agonist (Ligand A) acting on a spiking cell

<img src="Example.png">

### Important information
- The agonist has been applied 5 times at each concentration and the data has been saved in .csv files, one for each concentration (csv = comma seperated values, typical text files for storing data, can be opened in excel)
- The data was sampled at 50 kHz
- The agonist was applied for 1 second in each trial
- Quantify the average number of spikes at each conencentration
- Then fit a Hill curve to this data


In [ ]:
#Load the necessary libraries 
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import scipy.optimize as fit
import glob    
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({'font.size': 14, 'figure.figsize': [10, 8]})
# sets some default parameters for the size and font size of all plots

### Get a list of the file names that we will process

- use glob.glob( ) to create a **list** called "files" that contains the name of each file in the "Data sets" folder
- print some text to inform us how many concentrations were tested
- print the contents of the list to make sure it is ordered correctly
- you learnt how to do this in PreWork3

The file names in **"files"** are not in acscending order of concentration. We want them to be ordered in ascending concentration as this will make later analysis much easier. 

So we will modify the code so that the file names are **sorted**
- copy and paste your code form above in the cell below
- modify it so that glob.glob( ) is inside sorted( ), i.e. <code>files=sorted(glob.glob('____'))</code>


### You now have an ordered list of file names that we are going to process
--------
# Next load one of the files and inspect it

- make data = **np.genfromtxt(___ , delimiter=',')** and load the **last** file in the list, i.e. the strongest concentration
- do this by replacing the blank with files[ ] with the appropriate index
- print the **shape** of the data, i.e. how many rows and columns
- you learnt how to do this in PreWork1 & 2

- now make an xScale array to plot the data with the correct x scaling, use np.arange()
- recall that np.arange( ) wants startTime,endTime,interval
- you need the sample rate for this which is in the important information above
- you learnt how to do this in PreWork2

- and plot the first column in a new figure and inspect it

### Now we will detect how many spikes were in this trial. You can reuse the appropriate code from your pre-work book to do this..

- you may want to refer to your completed PreWork2 notebook
- remember that find_peaks has 2 outputs
- print the number of spikes in this trial

### Now loop through each trial doing the same thing
- create a **for** loop with i in range(data.shape[1]) to get the number of columns, refer to PreWork1 if needed
- paste you code from the above cell inside the for loop
- make sure to change the column index to 'i' in data
- make sure the print statement is also inside the loop

### Instead of printing the values we want to put them in an array so it is easy to calculate the mean

- so first create an array to hold these values, call it "spikes", you can use np.zeros( ) to generate an empty array, make it as long as there are columns using data.shape[1] inside the brackets
- then run the loop again but instead of printing, put the number of spikes into the index of spikes, i.e. <code>spikes[i]=len(peaks)</code>

- after the loop (i.e. not indented) print the mean number of spikes for this trial and the standard deviation. use <code>np.mean()</code> and <code>np.std()</code>

### We are now ready to do this for all the files

- first create 2 lists meanSpikes and SD, to store the mean and SD from each file respectively
- then create a for loop to loop file in files:
- inside this use np.genfromtxt to load each file, copy your code from above for this
- then paste the code you developed in the above cell inside this loop
- instead of print the mean and SD append them to their list, see PreWork1 for how to append to a list
- take care with your indentation!


#### Great! you have just looped through all the data, detected spikes and calculated the mean and SD for each trial!

You just need to plot these, first you need an array corresponding to concentrations
- the code below creates this for you

In [ ]:
concentration=np.array([0.0001,0.0003,0.001,0.003,0.01,0.03,0.1,0.3])

- use the following to plot your results:

<code>plt.errorbar(concentration,meanSpikes,yerr=SD,fmt='o');</code>

see details here (https://matplotlib.org/api/_as_gen/matplotlib.pyplot.errorbar.html)
- change x axis to log scale: <code>plt.xscale('log')</code>
- label the x and y axes, see PreWork2

## Now we will fit the the Hill equation to this data
<img src="HillEquation.png">

### To fit an equation to data we first need to "define" the equation as a python function 


In [ ]:
# remember a function receives inputs, in this case the parameters of the equation, i.e. EC50, n, etc
# importantly the the first input needs to be the dependent variable, in this case the concentration

def hillFunction(conc,Max,EC50,n):
    return Max/(1+(EC50/conc)**n)


### Now we want to fit this equation to our data to determine what the best value is for each of the parameters: EC50, n and max. We can use SciPy's optimize module which we have called "fit"
- follow the instruction in the below cell
- and then print the EC50 and the Hill coefficent
- check PreWork4 for a similar example if you get stuck

Details of the curvefit function can be found here: (https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html)

In [ ]:
# use fit.curve_fit to obtain the best parameters, we input the function we want to fit and the x and y data
# curve fit outputs 2 objects, the parameters which we want and the covariance which you can ignore.

params, paramsCoV = fit.curve_fit(hillFunction,concentration,meanSpikes)

# insert code to print the EC50 and hill coefficent "n"


### Finally we will plot our results with the fit we have just obtained
- first copy and paste the code for your previous plot of the mean and SD that you generated in a previous cell
- now generate a new concentration array with many more points, this will make our fit display as a smooth curve, call it "manyConcs", you can use np.arange( ), the start and end should be the lowest and highest concentration respectively, use 0.00001 for the interval
- then use plt.plot to add the curve to the graph.. for the y data you simply use the hill function with the appropriate index of "params". See PreWork4 for a similar example
- to finish off also print what the EC50 and Hill coefficent are